# SETI CNN using TF and Binary DS

In [1]:
import requests
import json
#import ibmseti
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pickle
import time
#!sudo pip install sklearn
from sklearn.metrics import confusion_matrix

In [2]:
!wget --output-document SETI.zip  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
!unzip -o SETI.zip
import SETI

--2017-05-16 10:04:10--  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip [following]
--2017-05-16 10:04:10--  https://ibm.ent.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/6dfZnXDJI8Vfgl2x3NesfY501F5ya-HT1Geu2Se0gzxT3eI8aOV8tFfRS5Rb5-NhCzquZai41Jyhdvms7rw3bargytDumEjJ0ADmAE0MgnZBqBQawqKwK2r9cnQUR14SeO9fxAL8k1erXBVwYvGXxaCKGhaEMW3gtuRyTjMqNaWYCeEvH8YbLzYPy-_mwfmLQJRfqg99hkejp2qAt0nPTy8qw8lSbuAjY3aj-V3_KcEMfrtHe5Ix8T1UsnL3B2gZhI-_fwYWl

### Download data

In [16]:
# The code was removed by DSX for sharing.

mkdir: cannot create directory ‘SETI’: File exists
--2017-05-16 07:59:03--  https://ibm.box.com/shared/static/ww315shk648d3mt3ljbyl1jjte8nkmvj.gz
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/ww315shk648d3mt3ljbyl1jjte8nkmvj.gz [following]
--2017-05-16 07:59:04--  https://ibm.ent.box.com/shared/static/ww315shk648d3mt3ljbyl1jjte8nkmvj.gz
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.27.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.27.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/vwmUfztdh4QmWdfYswL7eipaptVpmn6C5AbrT43-wIpUTiOUoi4V1aJB9bkOEERz4lhq3wOum84AqRmVO-dz4MWsGrsMEiwtDYJ84mEXN-tBp21pxTQmQ6uLflkGMOB8J3LOR3FgzNzf5FKqSncd9AixmdhP8CD50Dny4nx_wKcQ8La9mmcFuCvcGvgkIPgxMrU4GOMYo9sFh4sVLV01nMFHUnGWV4CLrzcrLNxng

### Load data

In [3]:
SETIds = SETI.read_data_sets('SETI/SETIds9/', one_hot=True, validation_size=0)
SETIds.train.num_examples

Extracting SETI/SETIds9/train-images-idx3-ubyte.gz
Extracting SETI/SETIds9/train-labels-idx1-ubyte.gz
Extracting SETI/SETIds9/test-images-idx3-ubyte.gz
Extracting SETI/SETIds9/test-labels-idx1-ubyte.gz


694

In [4]:
SETIds.train.images.shape

(694, 131072)

## CNN

In [5]:
# Parameters
learning_rate = 0.001
max_training_iters = 50
batch_size = 20
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 4 
dropout = 0.75 # Dropout, probability to keep units

height = 256 # height of the image in pixels -- 128
width = 512 # width of the image in pixels -- 1536
flat = width * height # number of pixels in one image  -196608
class_output = 4 # number of possible classifications for the problem

In [6]:
x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [7]:
x_image = tf.reshape(x, [-1,height,width,1]) 
x_image

<tf.Tensor 'Reshape:0' shape=(?, 256, 512, 1) dtype=float32>

#### Convolutional Layer 1

In [8]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
h_conv1 = tf.nn.relu(convolve1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer1= h_pool1
layer1

<tf.Tensor 'MaxPool:0' shape=(?, 128, 256, 32) dtype=float32>

#### Convolutional Layer 2

In [9]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs
convolve2= tf.nn.conv2d(layer1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2
h_conv2 = tf.nn.relu(convolve2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer2= h_pool2
layer2

<tf.Tensor 'MaxPool_1:0' shape=(?, 64, 128, 64) dtype=float32>

#### Fully Connected Layer 3

In [27]:
dim = layer2.get_shape().as_list()
dim

[None, 64, 128, 64]

In [29]:
dims= dim[1]*dim[2]*dim[3]
layer2_matrix = tf.reshape(layer2, [-1, dims])
W_fc1 = tf.Variable(tf.truncated_normal([dims, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) # need 1024 biases for 1024 outputs
fcl3=tf.matmul(layer2_matrix, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(fcl3)
layer3= h_fc1
layer3

<tf.Tensor 'Relu_3:0' shape=(?, 1024) dtype=float32>

In [30]:
keep_prob = tf.placeholder(tf.float32)
layer3_drop = tf.nn.dropout(layer3, keep_prob)

#### Layer 4

In [31]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, 4], stddev=0.1)) #1024 neurons
b_fc2 = tf.Variable(tf.constant(0.1, shape=[4])) # 10 possibilities for classes [0,1,2,3]
fcl4=tf.matmul(layer3_drop, W_fc2) + b_fc2
y_l4_conv= tf.nn.softmax(fcl4)

#### Training


In [32]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_l4_conv), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fcl4, labels=y_))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [33]:
correct_prediction = tf.equal(tf.argmax(y_l4_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [34]:
# Initializing the variables
init = tf.initialize_all_variables()


In [35]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step < max_training_iters:
    # Keep training until reach max iterations
        
        start = time.time()

        x_batch, y_batch = SETIds.train.next_batch(batch_size)
        # Run optimization op (backprop)
        #sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: dropout})

        end = time.time()

        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: x_batch,y_: y_batch,keep_prob: 1.})
            #train_accuracy = accuracy.eval(feed_dict={x:x_batch, y_: y_batch,  keep_prob: 0.5})

            print("Iter " + str(step) + \
                ", Training time= " + "{:.5f}".format(end - start) + \
                ", Minibatch Loss= " +  "{:.6f}".format(loss) +  \
                ", Training Accuracy= " + "{:.5f}".format(acc) )
        step += 1
    print("Optimization Finished!")
    
    X_test = SETIds.test.images
    y_test = SETIds.test.labels
    
    # Calculate accuracy for test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: X_test, y_: y_test, keep_prob: 1.}))
        
    # Find the labels of test set
    y = sess.run(tf.argmax(y_l4_conv,1), feed_dict={x: X_test, y_: y_test, keep_prob: 1.})
    
    # lets save kernels
    kernels_l1 = sess.run(tf.reshape(tf.transpose(W_conv1, perm=[2, 3, 0, 1]),[32,-1]))
    kernels_l2 = sess.run(tf.reshape(tf.transpose(W_conv2, perm=[2, 3, 0, 1]),[32*64,-1]))

ResourceExhaustedError: OOM when allocating tensor with shape[524288,1024]
	 [[Node: truncated_normal_4/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](truncated_normal_4/TruncatedNormal, truncated_normal_4/stddev)]]
Caused by op u'truncated_normal_4/mul', defined at:
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/src/bluemix_jupyter_bundle.v43/notebook/lib/python2.7/site-packages/i

## Evaluation

In [ ]:
y_ = np.argmax(y_test,1)
confusion_matrix(y_, y)

### Viz

In [ ]:
!wget --output-document utils1.py http://deeplearning.net/tutorial/code/utils.py
import utils1
from utils1 import tile_raster_images

In [ ]:
#from utils import tile_raster_images
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
image = Image.fromarray(tile_raster_images(kernels_l1, img_shape=(5, 5) ,tile_shape=(4, 8), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')  

In [17]:
image = Image.fromarray(tile_raster_images(kernels_l2, img_shape=(5, 5) ,tile_shape=(4, 12), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')  

NameError: name 'Image' is not defined